In [2]:
%load_ext autoreload
%autoreload 2

# Polybench Experimental Results

This notebook puts together how we run Polybench experiments using the `pb-flow` script, and how we process the run results to get the formatted table.

## Setup

We need to put the `python/` directory into the `PYTHONPATH`. 

In [3]:
import sys
import os
import pandas as pd

sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())))

In [4]:
import python.utils.polybench as pb_utils

In [5]:
len(pb_utils.discover_examples('./example/polybench'))

30

In [6]:
options = {
  "baseline": pb_utils.PbFlowOptions(
    pb_dir=os.path.join(pb_utils.get_project_root(), 'example', 'polybench'),
    work_dir=os.path.join(pb_utils.get_project_root(), 'tmp', 'phism', 'pb-flow.baseline'),
    job=30,
    polymer=False,
    cosim=True,
    debug=True,
    dataset='MINI',
    cleanup=False,
    loop_transforms=False,
    improve_pipelining=False,
  ),
  "polymer": pb_utils.PbFlowOptions(
    pb_dir=os.path.join(pb_utils.get_project_root(), 'example', 'polybench'),
    work_dir=os.path.join(pb_utils.get_project_root(), 'tmp', 'phism', 'pb-flow.polymer'),
    job=30,
    polymer=True,
    cosim=True,
    debug=True,
    dataset='MINI',
    cleanup=False,
    loop_transforms=True,
    max_span=1,
  ),
}

## Run `pb-flow`

In [30]:
pb_utils.pb_flow_runner(options["baseline"])

>>> Starting 30 jobs (work_dir=/mnt/ccnas2/bdp/rz3515/projects/phism/tmp/phism/pb-flow.baseline) ...
>>> Finished nussinov        elapsed: 128.315691 secs   Status: 0  Error: "No Error"
>>> Finished trmm            elapsed: 144.567948 secs   Status: 1  Error: "Phism syn failed."
>>> Finished floyd-warshall  elapsed: 155.321162 secs   Status: 0  Error: "No Error"
>>> Finished atax            elapsed: 204.790661 secs   Status: 1  Error: "Cosim failed."
>>> Finished gemver          elapsed: 222.608564 secs   Status: 1  Error: "Cosim failed."
>>> Finished symm            elapsed: 227.652830 secs   Status: 1  Error: "Cosim failed."
>>> Finished jacobi-1d       elapsed: 230.418896 secs   Status: 0  Error: "No Error"
>>> Finished syrk            elapsed: 232.207221 secs   Status: 1  Error: "Cosim failed."
>>> Finished bicg            elapsed: 240.552735 secs   Status: 0  Error: "No Error"
>>> Finished mvt             elapsed: 244.524898 secs   Status: 0  Error: "No Error"
>>> Finished gesummv

In [7]:
pb_utils.pb_flow_runner(options["polymer"])

>>> Starting 30 jobs (work_dir=/mnt/ccnas2/bdp/rz3515/projects/phism/tmp/phism/pb-flow.polymer) ...
>>> Finished nussinov        elapsed: 88.613960 secs   Status: 1  Error: "Phism syn failed."
>>> Finished floyd-warshall  elapsed: 138.800980 secs   Status: 0  Error: "No Error"
>>> Finished jacobi-1d       elapsed: 215.504993 secs   Status: 1  Error: "Cosim failed."
>>> Finished bicg            elapsed: 217.498600 secs   Status: 1  Error: "Cosim failed."
>>> Finished gemver          elapsed: 236.696349 secs   Status: 1  Error: "Cosim failed."
>>> Finished durbin          elapsed: 241.535897 secs   Status: 1  Error: "Cosim failed."
>>> Finished atax            elapsed: 244.781526 secs   Status: 0  Error: "No Error"
>>> Finished syrk            elapsed: 245.098366 secs   Status: 1  Error: "Cosim failed."
>>> Finished gesummv         elapsed: 249.772194 secs   Status: 0  Error: "No Error"
>>> Finished mvt             elapsed: 252.301086 secs   Status: 0  Error: "No Error"
>>> Finished trmm

## Results

Here we list all the results as a table.

In [8]:
baseline = pb_utils.to_pandas(pb_utils.process_pb_flow_result_dir(options['baseline'].work_dir, options['baseline']))
baseline

,name,phism_synth,tbgen_cosim,phism_cosim,latency,syn_latency,DSP_usage,FF_usage,LUT_usage,BRAM_18K_usage,URAM_usage,DSP_avail,FF_avail,LUT_avail,BRAM_18K_avail,URAM_avail
0,2mm,SUCCESS,SUCCESS,SUCCESS,99859,99859,15,2211,2665,0,0,220,106400,53200,280,0
1,3mm,SUCCESS,SUCCESS,SUCCESS,172846,172846,16,2700,3311,0,0,220,106400,53200,280,0
2,adi,SUCCESS,SUCCESS,SUCCESS,768301,768301,14,2995,3479,0,0,220,106400,53200,280,0
3,atax,SUCCESS,SUCCESS,COSIM_FAILED,None,4339,16,1942,3164,0,0,220,106400,53200,280,0
4,bicg,SUCCESS,SUCCESS,SUCCESS,12821,12821,15,1933,2179,0,0,220,106400,53200,280,0
5,cholesky,SUCCESS,SUCCESS,SUCCESS,216706,None,20,4065,4628,0,0,220,106400,53200,280,0
6,correlation,SUCCESS,SUCCESS,COSIM_FAILED,None,None,16,3668,5376,0,0,220,106400,53200,280,0
7,covariance,SUCCESS,SUCCESS,COSIM_FAILED,None,None,16,2749,4039,0,0,220,106400,53200,280,0
8,deriche,CANNOT_SYNTH,SUCCESS,NO_LOG,237664,None,None,None,None,None,None,None,None,None,None,None
9,doitgen,SUCCESS,SUCCESS,SUCCESS,94161,94161,14,1445,2189,0,0,220,106400,53200,280,0


In [9]:
polymer = pb_utils.to_pandas(pb_utils.process_pb_flow_result_dir(options['polymer'].work_dir, options['polymer']))
polymer

,name,phism_synth,tbgen_cosim,phism_cosim,latency,syn_latency,DSP_usage,FF_usage,LUT_usage,BRAM_18K_usage,URAM_usage,DSP_avail,FF_avail,LUT_avail,BRAM_18K_avail,URAM_avail
0,2mm,SUCCESS,SUCCESS,SUCCESS,29241,29241,25,3134,4281,6,0,220,106400,53200,280,0
1,3mm,SUCCESS,SUCCESS,SUCCESS,37933,None,22,4356,5181,6,0,220,106400,53200,280,0
2,adi,SUCCESS,SUCCESS,COSIM_FAILED,None,104701,39,6056,6896,8,0,220,106400,53200,280,0
3,atax,SUCCESS,SUCCESS,SUCCESS,3914,None,18,3352,3814,4,0,220,106400,53200,280,0
4,bicg,SUCCESS,SUCCESS,COSIM_FAILED,None,None,18,3352,3806,4,0,220,106400,53200,280,0
5,cholesky,SUCCESS,SUCCESS,COSIM_FAILED,None,None,29,3915,5225,32,0,220,106400,53200,280,0
6,correlation,SUCCESS,SUCCESS,COSIM_FAILED,None,None,17,5433,6523,12,0,220,106400,53200,280,0
7,covariance,SUCCESS,SUCCESS,SUCCESS,45205,None,14,3513,4459,10,0,220,106400,53200,280,0
8,deriche,SUCCESS,SUCCESS,COSIM_FAILED,237664,None,None,None,None,None,None,None,None,None,None,None
9,doitgen,SUCCESS,SUCCESS,SUCCESS,27381,27381,14,1743,2412,0,0,220,106400,53200,280,0


In [10]:
from collections import defaultdict

def get_latency(df, name):
  cosim_latency = df[df['name'] == name]['latency'].values[0]
  if cosim_latency:
    return cosim_latency, True
  synth_latency = df[df['name'] == name]['syn_latency'].values[0]
  return synth_latency, False

base_succ = pb_utils.filter_success(baseline)
poly_succ = pb_utils.filter_success(polymer)

names = sorted(list(set(base_succ['name']) & set(poly_succ['name'])))

data = defaultdict(list)

for name in names:
  data['name'].append(name)

  data['base_lat'].append(get_latency(base_succ, name)[0])
  data['poly_lat'].append(get_latency(poly_succ, name)[0])
  data['cosim'].append(get_latency(poly_succ, name)[1])

df = pd.DataFrame(data)
df['speedup'] = (df['base_lat'] / df['poly_lat'])
df

,name,base_lat,poly_lat,cosim,speedup
0,2mm,99859,29241,True,3.415034
1,3mm,172846,37933,True,4.556613
2,adi,768301,104701,False,7.338048
3,atax,4339,3914,True,1.108585
4,doitgen,94161,27381,True,3.438917
5,fdtd-2d,274141,675325,True,0.405939
6,floyd-warshall,432072,507721,True,0.851003
7,gemm,123881,123525,False,1.002882
8,gesummv,7223,2253,True,3.205948
9,heat-3d,84021,745950,True,0.112636


In [11]:
df['speedup'].mean()

2.798319663565662